In [1]:
import numpy as np
import tensorflow as tf
import cv2 as cv
import os 
import random
from os.path import join,exists
from glob import glob
from natsort import natsorted
from tensorflow.keras.utils import to_categorical

2023-01-25 17:55:14.745703: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-25 17:55:15.232556: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ros/foxy/opt/yaml_cpp_vendor/lib:/opt/ros/foxy/opt/rviz_ogre_vendor/lib:/opt/ros/foxy/lib/x86_64-linux-gnu:/opt/ros/foxy/lib:/usr/local/cuda-11.7/lib64${LD_LIBRARY_PATH:+:}
2023-01-25 17:55:15.232725: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object fi

: 

In [11]:
pose_path='../features/smarthome/rgb/densenet101/CO/crop'
rgb_path='../features/smarthome/skeleton/alphapose/CO'

In [15]:
classnames=os.listdir(rgb_path)

In [58]:
npy_files=natsorted(glob(rgb_path+'/**/*.npy'))
pose_files=natsorted(glob(pose_path+'/**/*.npy'))

In [59]:
combine_features=list(zip(pose_files,npy_files))
random.shuffle(combine_features)

In [62]:
combine_features[210]

('../features/smarthome/rgb/densenet101/CO/crop/Drink.Fromcup/Drink.Fromcup_p15_r08_v08_c02.npy',
 '../features/smarthome/skeleton/alphapose/CO/Drink.Fromcup/Drink.Fromcup_p15_r08_v08_c02.npy')

In [54]:
len(pose_files)

906

In [55]:
p,k=zip(*combine_features)

In [56]:
p[5]

'../features/smarthome/rgb/densenet101/CO/crop/Drink.Frombottle/Drink.Frombottle_p03_r00_v08_c05.npy'

In [57]:
k[5]

'../features/smarthome/skeleton/alphapose/CO/Drink.Frombottle/Drink.Frombottle_p03_r00_v06_c05.npy'

In [82]:
class FrameGenerator:

    def __init__(self,pose_path,rgb_path,training=True):

        self.pose_path=pose_path

        self.rgb_path=rgb_path
        
        self.training=training
        
        
        self.classnames=natsorted(os.listdir(self.rgb_path))
        self.num_class=len(self.classnames)
        

    
    def get_label(self,npz_file):
        classname=os.path.basename(npz_file).split('_')[0]
        return self.classnames.index(classname)

    def load_npy(self,npy_file):
        features=np.load(npy_file)
        return features

    def combine_inputs(self,X_pose, X_rgb, y):
        return {"input_1": X_pose, "input_2": X_rgb}, y

    def __call__(self):

        pose_npy_list=natsorted(glob(self.pose_path+'/**/*.npy'))
        rgb_npy_list=natsorted(glob(self.rgb_path+'/**/*.npy'))

        combine_features=list(zip(pose_npy_list,rgb_npy_list))
        

        if self.training:
            random.shuffle(combine_features)

        for pose_npy_file,rgb_npy_file in combine_features:
            pose_feature=self.load_npy(pose_npy_file)
            rgb_imgs=self.load_npy(rgb_npy_file)

            
            label=self.get_label(pose_npy_file)

            # feat_dict=self.combine_inputs(pose_feature,rgb_imgs,label)

            yield pose_feature,rgb_imgs,to_categorical(label,num_classes=self.num_class)

            # yield features,to_categorical(label,num_classes=self.num_class)

In [87]:
fg=FrameGenerator(pose_path=pose_path,rgb_path=rgb_path,training=True)

In [26]:
output_signature = (tf.TensorSpec(shape = (None,None,None, 3), dtype = tf.string),
                    tf.TensorSpec(shape = (31), dtype = tf.int16))

In [93]:
output_signature = (tf.TensorSpec(shape = (None,34), dtype = tf.float32),
                    tf.TensorSpec(shape = (None,None,None,3), dtype = tf.float32),
                    tf.TensorSpec(shape = (3), dtype = tf.int16))

In [94]:
ds=tf.data.Dataset.from_generator(fg,output_signature=output_signature)


In [95]:
ds1=ds.batch(1)

In [96]:
list(ds1.as_numpy_iterator())[0]

2023-01-25 15:59:09.006924: W tensorflow/core/framework/op_kernel.cc:1818] INVALID_ARGUMENT: TypeError: `generator` yielded an element of shape (32, 350, 350, 3) where an element of shape (None, 34) was expected.
Traceback (most recent call last):

  File "/home/cvlab/anaconda3/envs/vision/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 271, in __call__
    ret = func(*args)

  File "/home/cvlab/anaconda3/envs/vision/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "/home/cvlab/anaconda3/envs/vision/lib/python3.10/site-packages/tensorflow/python/data/ops/dataset_ops.py", line 1076, in generator_py_func
    raise TypeError(

TypeError: `generator` yielded an element of shape (32, 350, 350, 3) where an element of shape (None, 34) was expected.




InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_3_device_/job:localhost/replica:0/task:0/device:CPU:0}} TypeError: `generator` yielded an element of shape (32, 350, 350, 3) where an element of shape (None, 34) was expected.
Traceback (most recent call last):

  File "/home/cvlab/anaconda3/envs/vision/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 271, in __call__
    ret = func(*args)

  File "/home/cvlab/anaconda3/envs/vision/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "/home/cvlab/anaconda3/envs/vision/lib/python3.10/site-packages/tensorflow/python/data/ops/dataset_ops.py", line 1076, in generator_py_func
    raise TypeError(

TypeError: `generator` yielded an element of shape (32, 350, 350, 3) where an element of shape (None, 34) was expected.


	 [[{{node PyFunc}}]] [Op:IteratorGetNext]

2023-01-25 15:59:09.050091: W tensorflow/core/framework/op_kernel.cc:1818] INVALID_ARGUMENT: TypeError: `generator` yielded an element of shape (32, 350, 350, 3) where an element of shape (None, 34) was expected.
Traceback (most recent call last):

  File "/home/cvlab/anaconda3/envs/vision/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 271, in __call__
    ret = func(*args)

  File "/home/cvlab/anaconda3/envs/vision/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "/home/cvlab/anaconda3/envs/vision/lib/python3.10/site-packages/tensorflow/python/data/ops/dataset_ops.py", line 1076, in generator_py_func
    raise TypeError(

TypeError: `generator` yielded an element of shape (32, 350, 350, 3) where an element of shape (None, 34) was expected.




: 

In [25]:
tf.Variable('Hello')

<tf.Variable 'Variable:0' shape=() dtype=string, numpy=b'Hello'>